# Imports & Constants

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns
import statsmodels.api as sm

from scipy import stats
from matplotlib.colors import ListedColormap

In [4]:
from jupyter_utils import style, mean_std, display_test, display_group_test, show_corrtest_mask_corr

In [5]:
import warnings
warnings.filterwarnings(action='ignore', category=np.VisibleDeprecationWarning)
warnings.filterwarnings(action='ignore', message='All-NaN slice encountered')
warnings.filterwarnings(action='ignore', message='Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.')

In [6]:
sns.set_theme(style="whitegrid")

In [7]:
PATH = '/Users/galina.ryazanskaya/Downloads/thesis?/code?/'

In [8]:
PATH_FIG = '/Users/galina.ryazanskaya/Downloads/thesis?/figures/ru/'

# Load and merge resulting metrics with psychosocial data

In [9]:
df = pd.read_csv(PATH +'rus_merged_psychosocial_data.csv', index_col=0)
df = df[df.index.notnull()]
df.rename(columns={'dep.severity-1': 'dep.severity',
                  'HDRS-17.score-1': 'HDRS-17',
                   'panss-1-td': 'panss_td',
                   'panss-1-total': 'panss_total', 
                   'panss-n-1-total': 'panss_neg', 
                   'panss-o-1-total': 'panss_o',
                   'panss-p-1-total': 'panss_pos', 
                   'sans-1-total': 'sans',
                   'saps-ftd-1-total': 'saps_ftd', 
                   'sops-1-total': 'sops_total', 
                   'sops-c-total': 'sops_c',
                   'sops-d-total': 'sops_d', 
                   'sops-n-total': 'sops_n', 
                   'sops-p-total': 'sops_p'
                  }, inplace=True)
df['index'] = df.index
df.drop_duplicates(inplace=True, subset='index')
df.drop(columns=['index'], inplace=True)

In [10]:
res_df = pd.read_csv(PATH + 'processed_values/ru_both.tsv', sep='\t', index_col=0, header=[0, 1, 2])

### Filter out only the patients there are assesment scores for

In [11]:
# find datapoints missing psychiatric assessment to drop them
dfi = [i.replace('-', '').replace('S', 'PD1') for i in df.index]
df.index = dfi

rdfi = [i.split('_')[0].replace('S', 'PD1') for i in res_df.index]

missing_psy = set(rdfi).difference(set(dfi))
missing_psy

{'PN005', 'PN006', 'PN012', 'PN014', 'PN019', 'PN238', 'PN327'}

In [12]:
missing_psy = [i for i in res_df.index if i.split('_')[0] in missing_psy]
res_df.drop(missing_psy, inplace=True)

In [13]:
missing_text = set(dfi).difference(set(rdfi))
assert not missing_text

In [14]:
non_start_timepoint = [i for i in res_df.index if i.split('_')[-1] != '1']
res_df.drop(non_start_timepoint, inplace=True)

In [15]:
rdfi_filtered = [i.split('_')[0].replace('S', 'PD1') for i in res_df.index]
res_df.index = rdfi_filtered

## Explore task availability

In [16]:
raw = pd.read_csv(PATH+'rus_transcript_lex_by_task_with_dots.tsv', sep='\t', index_col=0)

In [17]:
ids_to_drop = [i for i in raw.index if i.split('_')[0] not in res_df.index]

In [18]:
raw.drop(index=ids_to_drop, inplace=True)

In [19]:
raw.index = [i.split('_')[0] for i in raw.index]

In [20]:
task_available = raw.applymap(lambda x: 1 if not pd.isna(x) else x)

In [21]:
task_available['diagnosis.type'] = df['diagnosis.type']

In [22]:
task_available.count()

adventure         99
bench             28
chair             75
party             35
present           81
sportsman         96
table             30
trip              40
winterday         40
diagnosis.type    49
dtype: int64

In [23]:
task_available.groupby('diagnosis.type').count()

,adventure,bench,chair,party,present,sportsman,table,trip,winterday
diagnosis.type,,,,,,,,,
dep,14,0,14,0,13,14,0,0,0
sz,30,0,17,0,21,28,0,1,0


In [24]:
task_available.groupby('diagnosis.type').count()[task_available.groupby('diagnosis.type').count() > 0].dropna(axis=1)

,adventure,chair,present,sportsman
diagnosis.type,,,,
dep,14,14,13,14
sz,30,17,21,28


In [25]:
TASKS = ['adventure', 'chair', 'present', 'sportsman']

In [26]:
def drop_person(row):
    for task in TASKS:
        if not pd.isna(row[task]):
            return False
    return True

In [27]:
ids_with_at_least_one_task = task_available[~task_available.apply(drop_person, axis=1)].index

In [28]:
df = df.loc[ids_with_at_least_one_task]

In [29]:
res_df = res_df.loc[ids_with_at_least_one_task, TASKS]

In [30]:
def task_data(df, task, keep_target=True, fill_synt=True):
    subset = df[task].dropna(axis=0, how='all')
    if fill_synt:
        subset['syntactic'] = subset['syntactic'].fillna(0.0)   # fills NAs in missing POS
    if keep_target:
        subset = pd.concat([subset, df['target'].loc[subset.index]], axis=1)
    return subset

In [31]:
def aplly_to_all_tasks(df, f, tasks=TASKS, to_df=True, *args, **kwargs):
    res = {}
    for task in tasks:
        data = task_data(df, task)
        res[task] = f(data, *args, **kwargs)
    if to_df:
        if all(isinstance(v, pd.Series) for v in res.values()):
            return pd.DataFrame(res)
        elif all(isinstance(v, pd.DataFrame) for v in res.values()):
            return pd.concat(list(res.values()), keys=list(res.keys()), names=['task'], axis=1)
        else:
            return res
    return res

### Analyze task lengths

In [32]:
compare_task_lens_df = pd.DataFrame(columns=TASKS)

In [33]:
compare_task_lens_df.loc['mean_sent_len'] = res_df[[(task, 'syntactic', 'mean_sent_len') for task in TASKS]].mean().droplevel([1, 2])
compare_task_lens_df.loc['n_sents'] = res_df[[(task, 'syntactic', 'n_sents') for task in TASKS]].mean().droplevel([1, 2])
compare_task_lens_df.loc['n_words'] =  res_df[[(task, 'lexical', 'n_words') for task in TASKS]].mean().droplevel([1, 2])

In [34]:
compare_task_lens_df

,adventure,chair,present,sportsman
mean_sent_len,7.165963,8.668295,9.724067,7.770671
n_sents,19.393939,18.506667,14.000000,17.635417
n_words,137.979798,159.866667,127.802469,131.322917


# Explore descriptive statistics

In [35]:
def fill_diagnosis_type(row):
    dt = row['diagnosis.type']
    if not pd.isna(dt):
        return dt
    else:
        if not pd.isna(row['td.severity']):
            return 'control_psy'
        else:
            return 'control'

In [36]:
df['diagnosis.type'] = df.apply(fill_diagnosis_type, axis=1)

In [37]:
sz = df[df['diagnosis.type'] == 'sz']
dep = df[df['diagnosis.type'] == 'dep']
control = df[df['diagnosis.group'] == 'control']
control_psy = df[df['diagnosis.type'] == 'control_psy']

In [38]:
df['diagnosis.group'].value_counts()

control    102
patient     49
Name: diagnosis.group, dtype: int64

In [39]:
df['diagnosis.type'].value_counts()

control        72
sz             31
control_psy    30
dep            18
Name: diagnosis.type, dtype: int64

In [40]:
df['dep.scale'].value_counts()

HDRS    51
QIDS    28
Name: dep.scale, dtype: int64

In [41]:
df['td.scales'].value_counts()

PANSS       28
SCL-90-R    28
SAPS        21
Name: td.scales, dtype: int64

In [42]:
df.groupby('diagnosis.type')[['diagnosis_code', 'diagnosis_eng']].value_counts()

diagnosis.type  diagnosis_code  diagnosis_eng                                    
dep             F31             bipolar.affective.disorder                            6
                F60.31          borderline.personality.disorder                       3
                F31.4           bipolar.affective.disorder.severe                     2
                F31.5           bipolar.affective.disorder.severe.psychotic           2
                F33             recurrent.depressive.disorder                         2
                F32.1           depressive.episode.moderate                           1
                F33.3           recurrent.depressive.disorder.severe.psychotic        1
                F60             personality.disorder                                  1
sz              F20             schizophrenia                                        20
                F25             schizoaffective.disorder                              8
                F21             schizo

### Target scale interactions

In [43]:
target_cols = ['sex', 'age', 'education.years', 
               'diagnosis.group', 'diagnosis.type',
               'dep.severity', 'td.severity']

In [44]:
panss_cols = ['panss_td', 'panss_total', 'panss_neg', 'panss_pos', 'panss_o']
sans_cols = [col for col in df.columns if col.startswith('sans')]
saps_cols = [col for col in df.columns if col.startswith('saps')]
sops_cols = [col for col in df.columns if col.startswith('sops')]

In [45]:
numeric_target = ['education.years', 'dep.severity', 'td.severity'] + panss_cols

In [46]:
# edu years - dep severity panss o
# dep severity - panss o total td severity
style(df[df['diagnosis.type'] != 'control'][numeric_target].corr())

,education.years,dep.severity,td.severity,panss_td,panss_total,panss_neg,panss_pos,panss_o
education.years,1.000000,-0.331386,-0.136782,-0.259404,-0.296650,-0.164954,-0.218234,-0.407709
dep.severity,-0.331386,1.000000,0.327470,0.173319,0.345454,0.174098,0.083666,0.597293
td.severity,-0.136782,0.327470,1.000000,0.911251,0.811338,0.692825,0.835814,0.708642
panss_td,-0.259404,0.173319,0.911251,1.000000,0.858317,0.756787,0.932452,0.695700
panss_total,-0.296650,0.345454,0.811338,0.858317,1.000000,0.922568,0.886422,0.885839
panss_neg,-0.164954,0.174098,0.692825,0.756787,0.922568,1.000000,0.788799,0.673327
panss_pos,-0.218234,0.083666,0.835814,0.932452,0.886422,0.788799,1.000000,0.687209
panss_o,-0.407709,0.597293,0.708642,0.695700,0.885839,0.673327,0.687209,1.000000


In [47]:
s, t  = display_test(df, numeric_target, 'td.severity', stats.pearsonr, stat_name='r')
# t['p'] = t['p'] * (len(numeric_target) - 1)
t.sort_values([f'abs_r', 'p'], ascending=False)['r']

panss_td           0.911251
panss_pos          0.835814
panss_total        0.811338
panss_o            0.708642
panss_neg          0.692825
dep.severity        0.32747
education.years   -0.136782
td.severity             NaN
Name: r, dtype: object

### SZ

In [48]:
sz['sex'].value_counts()

female    25
male       6
Name: sex, dtype: int64

In [49]:
mean_std(sz, target_cols + panss_cols)

/Users/galina.ryazanskaya/Downloads/thesis?/code?/jupyter_utils.py:118: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  s_mean, s_std = data.mean().round(2), data.std().round(2)


,age,education.years,dep.severity,td.severity,panss_td,panss_total,panss_neg,panss_pos,panss_o
value,27.13 (7.14),13.32 (2.41),0.58 (0.85),0.84 (0.73),10.03 (3.74),69.79 (16.13),22.93 (8.59),15.9 (4.92),30.97 (8.42)


In [50]:
mean_std(sz, target_cols + panss_cols, 'sex')

,age,education.years,dep.severity,td.severity,panss_td,panss_total,panss_neg,panss_pos,panss_o
sex,,,,,,,,,
female,27.8 (7.53),13.56 (2.48),0.72 (0.89),0.8 (0.76),9.43 (3.62),69.13 (15.38),22.52 (7.79),15.3 (4.91),31.3 (9.08)
male,24.33 (4.72),12.33 (1.97),0.0 (0.0),1.0 (0.63),12.33 (3.56),72.33 (20.16),24.5 (11.93),18.17 (4.67),29.67 (5.65)


In [51]:
r = mean_std(sz, ['dep.severity', 'td.severity'] + panss_cols)
r.loc['max'] = pd.Series([4, 4, 28, 210, 49, 49, 112], index=r.columns)
r.loc['min'] = pd.Series([0, 0, 2, 30, 7, 7, 16], index=r.columns)
r.loc['share'] = r.loc['value'].apply(lambda x: float(x.split(' ')[0])) / r.loc['max'] 
r

,dep.severity,td.severity,panss_td,panss_total,panss_neg,panss_pos,panss_o
value,0.58 (0.85),0.84 (0.73),10.03 (3.74),69.79 (16.13),22.93 (8.59),15.9 (4.92),30.97 (8.42)
max,4,4,28,210,49,49,112
min,0,0,2,30,7,7,16
share,0.145,0.21,0.358214,0.332333,0.467959,0.32449,0.276518


### Relative symptom severity (positive vs negative symptoms)  sz

In [52]:
fig, axes = plt.subplots(6, 1, figsize=(10, 8))
fig.suptitle('comparison between psychiatric scales')
sns.boxplot(x=sz['td.severity'], ax=axes[0], color='tab:orange')
axes[0].set_xlim(0, 4)
axes[0].set_ylabel('td.severity', rotation='horizontal', ha='right')
sns.boxplot(x=sz['dep.severity'], ax=axes[1])
axes[1].set_xlim(0, 4)
axes[1].set_ylabel('dep.severity', rotation='horizontal', ha='right')
sns.boxplot(x=sz['panss_pos'], ax=axes[2], color='tab:orange')
axes[2].set_xlim(7, 49)
axes[2].set_ylabel('panss_pos', rotation='horizontal', ha='right')
sns.boxplot(x=sz['panss_neg'], ax=axes[3])
axes[3].set_xlim(7, 49)
axes[3].set_ylabel('panss_neg', rotation='horizontal', ha='right')
sns.boxplot(x=sz['panss_o'], ax=axes[4], color='grey')
axes[4].set_xlim(16, 112)
axes[4].set_ylabel('panss_o', rotation='horizontal', ha='right')
sns.boxplot(x=sz['panss_total'], ax=axes[5], color='grey')
axes[5].set_xlim(30, 210)
axes[5].set_ylabel('panss_total', rotation='horizontal', ha='right')
axes[5].set_xlabel('score')
for ax in axes[:5]:
    ax.set_xlabel('')
fig.tight_layout();
plt.savefig(f'{PATH_FIG}psychiatric.png', dpi=150, bbox_inches = 'tight')
plt.close(fig)

### dep

In [53]:
dep['sex'].value_counts()

female    18
Name: sex, dtype: int64

In [54]:
mean_std(dep, target_cols + panss_cols)

/Users/galina.ryazanskaya/Downloads/thesis?/code?/jupyter_utils.py:118: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  s_mean, s_std = data.mean().round(2), data.std().round(2)


,age,education.years,dep.severity,td.severity,panss_td,panss_total,panss_neg,panss_pos,panss_o
value,20.89 (3.71),12.67 (1.94),0.56 (0.62),0.06 (0.24),4.42 (0.9),37.92 (5.89),8.31 (1.97),8.46 (1.94),21.15 (3.58)


In [55]:
dep['panss_o'].count()

13

### Relative symptom severity (positive vs negative symptoms)  dep

In [56]:
fig, axes = plt.subplots(11, 1, figsize=(15, 10))
fig.suptitle('comparison between psychiatric scales for depression group')
sns.boxplot(x=dep['td.severity'], ax=axes[0], color='tab:orange')
axes[0].set_xlim(0, 4)

axes[0].set_ylabel('td.severity', rotation='horizontal', ha='right')
sns.boxplot(x=dep['dep.severity'], ax=axes[1])
axes[1].set_xlim(0, 4)

axes[1].set_ylabel('dep.severity', rotation='horizontal', ha='right')
sns.boxplot(x=dep['panss_pos'], ax=axes[2], color='tab:orange')
axes[2].set_xlim(7, 49)

axes[2].set_ylabel('panss_pos', rotation='horizontal', ha='right')
sns.boxplot(x=dep['panss_neg'], ax=axes[3])
axes[3].set_xlim(7, 49)

axes[3].set_ylabel('panss_neg', rotation='horizontal', ha='right')
sns.boxplot(x=dep['panss_o'], ax=axes[4], color='grey')
axes[4].set_xlim(16, 112)

axes[4].set_ylabel('panss_o', rotation='horizontal', ha='right')
sns.boxplot(x=dep['panss_total'], ax=axes[5], color='grey')
axes[5].set_xlim(30, 210)

axes[5].set_ylabel('panss_total', rotation='horizontal', ha='right')
sns.boxplot(x=dep['sops_p'], ax=axes[6], color='tab:orange')
axes[6].set_xlim(0, 25)
axes[6].set_xlabel('')
axes[6].set_ylabel('sops_p', rotation='horizontal', ha='right')

sns.boxplot(x=dep['sops_n'], ax=axes[7])
axes[7].set_xlim(0, 30)
axes[7].set_xlabel('')
axes[7].set_ylabel('sops_n', rotation='horizontal', ha='right')

sns.boxplot(x=dep['sops_d'], ax=axes[8], color='tab:orange')
axes[8].set_xlim(0, 20)
axes[8].set_xlabel('')
axes[8].set_ylabel('sops_d', rotation='horizontal', ha='right')

sns.boxplot(x=dep['sans'], ax=axes[9])
axes[9].set_xlim(0, 160)
axes[9].set_xlabel('')
axes[9].set_ylabel('sans', rotation='horizontal', ha='right')

sns.boxplot(x=dep['saps_ftd'], ax=axes[10], color='tab:orange')
axes[10].set_xlim(0, 120)
axes[10].set_ylabel('saps', rotation='horizontal', ha='right')
axes[10].set_xlabel('score')

for ax in axes[:10]:
    ax.set_xlabel('')


fig.tight_layout();
plt.savefig(f'{PATH_FIG}psychiatric_dep.png', dpi=150, bbox_inches = 'tight')
plt.close(fig)

### Controls & Controls_Psy

In [57]:
control['sex'].value_counts()

female    75
male      27
Name: sex, dtype: int64

In [58]:
mean_std(control, target_cols)

/Users/galina.ryazanskaya/Downloads/thesis?/code?/jupyter_utils.py:118: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  s_mean, s_std = data.mean().round(2), data.std().round(2)


,age,education.years,dep.severity,td.severity
value,39.75 (19.15),15.74 (2.57),0.0 (0.0),0.0 (0.0)


In [59]:
mean_std(control, target_cols, 'sex')

,age,education.years,dep.severity,td.severity
sex,,,,
female,38.25 (18.85),15.39 (2.39),0.0 (0.0),0.0 (0.0)
male,43.93 (19.73),16.7 (2.83),0.0 (0.0),0.0 (0.0)


In [60]:
control_psy['sex'].value_counts()

female    26
male       4
Name: sex, dtype: int64

In [61]:
mean_std(control_psy, target_cols + panss_cols)

/Users/galina.ryazanskaya/Downloads/thesis?/code?/jupyter_utils.py:118: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  s_mean, s_std = data.mean().round(2), data.std().round(2)


,age,education.years,dep.severity,td.severity,panss_td,panss_total,panss_neg,panss_pos,panss_o
value,25.0 (7.4),15.43 (2.13),0.0 (0.0),0.0 (0.0),4.36 (1.0),30.77 (1.54),7.23 (0.53),7.23 (0.61),16.32 (0.95)


In [62]:
mean_std(control_psy, target_cols + panss_cols, 'sex')

,age,education.years,dep.severity,td.severity,panss_td,panss_total,panss_neg,panss_pos,panss_o
sex,,,,,,,,,
female,25.42 (7.8),15.42 (1.7),0.0 (0.0),0.0 (0.0),4.4 (1.05),30.85 (1.6),7.25 (0.55),7.25 (0.64),16.35 (0.99)
male,22.25 (3.3),15.5 (4.43),0.0 (0.0),0.0 (0.0),4.0 (0.0),30.0 (0.0),7.0 (0.0),7.0 (0.0),16.0 (0.0)


In [63]:
control.groupby('sex')['panss_total'].count()

sex
female    20
male       2
Name: panss_total, dtype: int64

# Test for group differences

### age

In [64]:
stats.ttest_ind(control['age'], sz['age'], nan_policy='omit')

Ttest_indResult(statistic=3.5871243575285408, pvalue=0.00047085918746073396)

In [65]:
stats.ttest_ind(control_psy['age'], sz['age'], nan_policy='omit')

Ttest_indResult(statistic=-1.143255849283786, pvalue=0.2575501479174914)

In [66]:
stats.ttest_ind(control_psy['age'], dep['age'], nan_policy='omit')

Ttest_indResult(statistic=2.1899209175114764, pvalue=0.033638603323606325)

### education years

In [67]:
stats.ttest_ind(control['education.years'], sz['education.years'], nan_policy='omit')

Ttest_indResult(statistic=4.653034980872723, pvalue=7.949204742694471e-06)

In [68]:
stats.ttest_ind(control_psy['education.years'], sz['education.years'], nan_policy='omit')

Ttest_indResult(statistic=3.618089157043871, pvalue=0.0006167702505040491)

In [69]:
stats.ttest_ind(control_psy['education.years'], dep['education.years'], nan_policy='omit')

Ttest_indResult(statistic=4.502842757884748, pvalue=4.561269295083997e-05)

### sex

In [70]:
a = 0.05

In [71]:
s_t_sex, res_t_sex = display_group_test(control, numeric_target, 'sex', stats.ttest_ind, stat_name='t', alpha=a)
style(res_t_sex)

,t,p,sig,abs_t
education.years,2.321636,0.022302,True,2.321636
dep.severity,nan,nan,False,nan
td.severity,nan,nan,False,nan
panss_td,-0.528886,0.602706,False,0.528886
panss_total,-0.735628,0.470497,False,0.735628
panss_neg,-0.628695,0.536664,False,0.628695
panss_pos,-0.541530,0.594124,False,0.541530
panss_o,-0.490038,0.629439,False,0.490038


In [72]:
s_t_sex, res_t_sex = display_group_test(control_psy, numeric_target, 'sex', stats.ttest_ind, stat_name='t', alpha=a)
style(res_t_sex)

,t,p,sig,abs_t
education.years,0.066128,0.947746,False,0.066128
dep.severity,nan,nan,False,nan
td.severity,nan,nan,False,nan
panss_td,-0.528886,0.602706,False,0.528886
panss_total,-0.735628,0.470497,False,0.735628
panss_neg,-0.628695,0.536664,False,0.628695
panss_pos,-0.541530,0.594124,False,0.541530
panss_o,-0.490038,0.629439,False,0.490038


In [73]:
s_t_sex, res_t_sex = display_group_test(sz, numeric_target, 'sex', stats.ttest_ind, stat_name='t', alpha=a)
style(res_t_sex)

,t,p,sig,abs_t
education.years,-1.122772,0.270744,False,1.122772
dep.severity,-1.954622,0.060329,False,1.954622
td.severity,0.592289,0.558247,False,0.592289
panss_td,1.753714,0.090826,False,1.753714
panss_total,0.426912,0.672829,False,0.426912
panss_neg,0.495589,0.624194,False,0.495589
panss_pos,1.282777,0.210473,False,1.282777
panss_o,-0.418000,0.679253,False,0.418000


### correlation between target variables

In [74]:
x, y = display_test(df, numeric_target, 'age', stats.pearsonr, stat_name='r', alpha=a)
y[y['abs_r']> 0.3].sort_values('abs_r', ascending=False)

,r,p,sig,abs_r
education.years,0.468966,0.0,True,0.468966
panss_neg,0.383708,0.001749,True,0.383708


In [75]:
x, y = display_test(df, numeric_target, 'education.years', stats.pearsonr, stat_name='r', alpha=a)
y[y['abs_r']> 0.3].sort_values('abs_r', ascending=False)

,r,p,sig,abs_r
panss_o,-0.407709,0.000826,True,0.407709
dep.severity,-0.331386,0.002852,True,0.331386


### Relative symptom severity (positive vs negative symptoms) by group

In [76]:
fig, axes = plt.subplots(3, 3, figsize=(15, 10))
fig.suptitle('PANSS score distribution')

sns.histplot(sz['panss_total'], ax=axes[0, 0], binwidth=5)
axes[0, 0].set_xlim(30, 95)
sns.histplot(dep['panss_total'], ax=axes[1, 0], binwidth=5)
axes[1, 0].set_xlim(30, 95)
sns.histplot(control_psy['panss_total'], ax=axes[2, 0], binwidth=5)
axes[2, 0].set_xlim(30, 95)
sns.histplot(sz['panss_neg'], ax=axes[0, 1], binwidth=2)
axes[0, 1].set_xlim(7, 45)
sns.histplot(dep['panss_neg'], ax=axes[1, 1], binwidth=2)
axes[1, 1].set_xlim(7, 45)
sns.histplot(control_psy['panss_neg'], ax=axes[2, 1], binwidth=2)
axes[2, 1].set_xlim(7, 45)
sns.histplot(sz['panss_pos'], ax=axes[0, 2], binwidth=2)
axes[0, 2].set_xlim(7, 30)
sns.histplot(dep['panss_pos'], ax=axes[1, 2], binwidth=2)
axes[1, 2].set_xlim(7, 30)
sns.histplot(control_psy['panss_pos'], ax=axes[2, 2], binwidth=2)
axes[2, 2].set_xlim(7, 30)

for ax, col in zip(axes[0], ('total', 'positive', 'negative')):
    ax.set_title(col)

for ax, row in zip(axes[:,0], ('NAP', 'Dep', 'HC')):
    ax.set_ylabel(row, rotation=0, size='large')
plt.close(fig)

In [77]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('severity')


sns.histplot(sz['dep.severity'], ax=axes[0, 0])
sns.histplot(dep['dep.severity'], ax=axes[1, 0])
sns.histplot(sz['td.severity'], ax=axes[0, 1])
sns.histplot(dep['td.severity'], ax=axes[1, 1])
axes[1, 1].set_xlim(0, 3)

for ax, col in zip(axes[0], ('Dep', 'TD')):
    ax.set_title(col)

for ax, row in zip(axes[:,0], ('NAP', 'Dep')):
    ax.set_ylabel(row, rotation=0, size='large')
plt.close(fig)

# Merge psychoscial data to scores

In [78]:
merge_df = res_df.copy()
for col in target_cols + panss_cols:
    merge_df[('target', 'target', col)] = df[col]

In [79]:
cols_tasks = res_df['sportsman'].columns
cols_LM = [col for col in res_df['sportsman'] if col[0] == 'LM']
cols_synt = [col for col in res_df['sportsman'] if col[0] == 'syntactic']
cols_lex = [col for col in res_df['sportsman'] if col[0] == 'lexical']
cols_graph = [col for col in res_df['sportsman'] if col[0] == 'graph']

In [80]:
POS_to_use = ('ADJ', 'ADV', 'AUX', 'CCONJ', 'DET','NOUN', 'PRON', 'PROPN', 'SCONJ', 'VERB', 'PART')
pos = set([x[1] for x in cols_synt if x[1].isupper()])

In [81]:
pos_cols_to_drop = [(task, c[0], c[1]) for task in TASKS for c in cols_synt if c[1].isupper() and c[1] not in POS_to_use and (task, c[0], c[1]) in merge_df.columns]

In [82]:
merge_df.drop(columns=pos_cols_to_drop, inplace=True)

In [83]:
cols_synt = [col for col in merge_df['chair'] if col[0] == 'syntactic']
cols_tasks = cols_synt + cols_LM + cols_lex + cols_graph

In [84]:
merge_df.to_csv('/Users/galina.ryazanskaya/Downloads/thesis?/code?/processed_values/ru_all.csv')

### for-task functions

In [85]:
def corr(df, target):
    return df[~pd.isnull(df[target])].corr()[target]

In [86]:
def corr_thresh(df, target, thresh=0.3, drop_target=True):
    corr_tgt = corr(df, target)
    if drop_target:
        corr_tgt.drop('target', inplace=True)
    return corr_tgt[abs(corr_tgt) >= thresh]

In [87]:
def ttest(df, test_columns, group):
    s_t, res_t = display_group_test(df, test_columns, group, stats.ttest_ind, stat_name='t', alpha=a)
    return res_t[['t', 'p']]

# Analyze the asscoiations with the control factors

In [88]:
def test_corr_sig(df, cols, target_col, mult=len(TASKS)):
    s, t = display_test(df, cols, target_col, stats.pearsonr, stat_name='r')
    t['p'] = t['p'] * (len(t) * mult)
    t['corr_sig'] = t['p'] < 0.05
    t['p'] = t['p'] / (len(t) * mult)
    t.drop(columns=['sig'], inplace=True)
    return t.sort_values([f'abs_r', 'p'], ascending=False)

In [89]:
tgt_tsk = [('target', c) for c in ('panss_total', 'panss_neg', 'panss_pos', 'panss_o', 'dep.severity', 'td.severity', 'age', 'education.years')]

### t-tests

In [90]:
def t_test_corr_sig(df, cols, target_col, mult=len(TASKS)):
    s, t = display_group_test(df, cols, target_col, stats.ttest_ind, stat_name='t', alpha=a)
    t['p'] = t['p'] * (len(t) * mult)
    t['corr_sig'] = t['p'] < 0.05
    t['p'] = t['p'] / (len(t) * mult)
    t.drop(columns=['sig'], inplace=True)
    return t.sort_values([f'abs_t', 'p'], ascending=False)

In [91]:
for task in TASKS:
    print(task)
    tres = t_test_corr_sig(task_data(merge_df, task, keep_target=True), cols_tasks, ('target', 'sex'))
    tr = tres[tres['corr_sig'] == True]
    if len(tr) > 0:
        print(tr)

adventure
chair
present
sportsman


### r correlations

In [92]:
for task in TASKS:
    print(task)
    tres = test_corr_sig(task_data(merge_df, task), tgt_tsk, ('syntactic', 'mean_sent_len'))
    tr = tres[tres['corr_sig'] == True]
    if len(tr) > 0:
        print(tr)

adventure
chair
present
sportsman


In [93]:
for task in TASKS:
    print(task)
    tres = test_corr_sig(task_data(merge_df, task), cols_tasks, ('target', 'age'))
    tr = tres[tres['corr_sig'] == True]
    if len(tr) > 0:
        print(tr)

adventure
chair
present
sportsman


In [94]:
for task in TASKS:
    print(task)
    tres = test_corr_sig(task_data(merge_df, task), cols_tasks, ('target', 'education.years'))
    tr = tres[tres['corr_sig'] == True]
    if len(tr) > 0:
        print(tr)

adventure
chair
present
sportsman


In [95]:
control_psy = merge_df[merge_df[('target', 'target', 'diagnosis.group')] == 'control_psy']

In [96]:
style(aplly_to_all_tasks(control_psy, corr_thresh, target=('target', 'age'), thresh=0.4))

,,adventure,chair,present,sportsman


In [97]:
style(aplly_to_all_tasks(control_psy, corr_thresh, target=('target', 'education.years'), thresh=0.4))

,,adventure,chair,present,sportsman


# Analyze lengths

In [98]:
c_map = {'control': 0, 'control_psy': 1, 'dep': 2, 'sz': 3}

In [99]:
for task in TASKS:
    print(task)
    for col in [('lexical', 'n_words'), ('syntactic', 'n_sents'), ('syntactic', 'mean_sent_len')]:
        tdf = task_data(merge_df, task, keep_target=True)
        tdf = tdf[tdf['target']['diagnosis.type'] != 'control']
        print('\t', col, f'{tdf[col].mean().round(1)} ({tdf[col].std().round(1)})')
        if col != ('lexical', 'n_words'):
            r = stats.pearsonr(tdf[col], tdf[('lexical', 'n_words')]).statistic
            print('\t\t', col, r.round(2))

adventure
	 ('lexical', 'n_words') 129.8 (82.1)
	 ('syntactic', 'n_sents') 18.7 (11.0)
		 ('syntactic', 'n_sents') 0.93
	 ('syntactic', 'mean_sent_len') 6.9 (1.5)
		 ('syntactic', 'mean_sent_len') 0.36
chair
	 ('lexical', 'n_words') 168.4 (120.9)
	 ('syntactic', 'n_sents') 20.4 (15.0)
		 ('syntactic', 'n_sents') 0.95
	 ('syntactic', 'mean_sent_len') 8.1 (1.6)
		 ('syntactic', 'mean_sent_len') 0.38
present
	 ('lexical', 'n_words') 140.8 (107.7)
	 ('syntactic', 'n_sents') 15.1 (12.7)
		 ('syntactic', 'n_sents') 0.95
	 ('syntactic', 'mean_sent_len') 9.9 (4.7)
		 ('syntactic', 'mean_sent_len') 0.08
sportsman
	 ('lexical', 'n_words') 134.4 (86.0)
	 ('syntactic', 'n_sents') 17.7 (10.1)
		 ('syntactic', 'n_sents') 0.93
	 ('syntactic', 'mean_sent_len') 7.8 (2.2)
		 ('syntactic', 'mean_sent_len') 0.11


In [100]:
def plot_scatter_corr(x, y, x_name, y_name, ax, title=None, c=None, 
                      cmap_colors=('steelblue', 'mediumseagreen', 'gold'), #'indigo', 
                      classes=('control_psy', 'dep', 'NAP'),
                         xlim=None, ylim=None): #'control',
    r = stats.pearsonr(x, y).statistic
    X = sm.add_constant(x)
    model = sm.OLS(y, X).fit()
    ypred = model.predict(X)
    if xlim:
        ax.set_xlim(*xlim)
    if ylim:
        ax.set_ylim(*ylim)

#     ax.plot(x, ypred, color='red', label=f'coef: {model.params[1]:.2f} \ncorr: {r:.2f}')
    ax.axline((x[0], ypred[0]), (x[1], ypred[1]),  color='red') #slope=r,

    
    if c is not None:
        cmap = ListedColormap(cmap_colors)
        scatter = ax.scatter(x, y, c=c, cmap=cmap)
        ax.legend(handles=scatter.legend_elements()[0], labels=classes)
    else:
        scatter = ax.scatter(x, y)
    ax.set_xlabel(x_name)
    ax.set_ylabel(y_name)

In [101]:
def plot_task(df, task, axes, xlims=(None, None), ylims=(None, None), xmin=(0, 4), ymins=(0, 0), force_no_xnames=False):
    tdf = task_data(df, task, keep_target=True)
    tdf = tdf[tdf['target']['diagnosis.type'] != 'control']
    c = [c_map[label] for label in tdf['target']['diagnosis.type']]
    x_name= '' if force_no_xnames else 'n_sents'
    plot_scatter_corr(tdf[('syntactic', 'n_sents')], tdf[('lexical', 'n_words')],
                     x_name=x_name, y_name='n_words', ax=axes[0], c=c, 
                      xlim=(xmin[0], xlims[0]), ylim=(ymins[0], ylims[0]))
    x_name= '' if force_no_xnames else 'mean_sent_len'
    plot_scatter_corr(tdf[('syntactic', 'mean_sent_len')], tdf[('lexical', 'n_words')],
                     x_name=x_name, y_name='', ax=axes[1], c=c, 
                      xlim=(xmin[1], xlims[1]), ylim=(ymins[1], ylims[1]))

In [102]:
fig = plt.figure(constrained_layout=True, figsize=(10, 20))
fig.suptitle('number of words against number of sentences and mean sentence length')

# create 4x1 subfigs
subfigs = fig.subfigures(nrows=4, ncols=1)
for i, subfig in enumerate(subfigs):
    task = TASKS[i]
    subfig.supylabel(f'{task} task')
    # create 1x2 subplots per subfig
    axs = subfig.subplots(nrows=1, ncols=2, sharey=True)
    plot_task(merge_df, task, axs, xlims=(100, 40), ylims=(800, 800), force_no_xnames=i < 3)
plt.savefig(f'{PATH_FIG}_n_words_n_sents.png', dpi=150, bbox_inches = 'tight')
plt.close(fig)